# Análise Exploratória de Dados

In [1]:
# importando bibliotecas
import os # manipulação de pastas
import warnings # exibe avisos
import numpy as np # manipulação numérica e matricial
import pandas as pd # manipulação de tabelas
import seaborn as sns # geração de gráficos
import matplotlib.pyplot as plt # geração de gráficos

warnings.filterwarnings('ignore')

In [2]:
# importando conjunto de dados
df = pd.read_csv('../data/raw/google_ads_data.csv')

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2600 entries, 0 to 2599
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Ad_ID            2600 non-null   object 
 1   Campaign_Name    2600 non-null   object 
 2   Clicks           2488 non-null   float64
 3   Impressions      2546 non-null   float64
 4   Cost             2503 non-null   object 
 5   Leads            2552 non-null   float64
 6   Conversions      2526 non-null   float64
 7   Conversion Rate  1974 non-null   float64
 8   Sale_Amount      2461 non-null   object 
 9   Ad_Date          2600 non-null   object 
 10  Device           2600 non-null   object 
 11  Keyword          2600 non-null   object 
dtypes: float64(5), object(7)
memory usage: 243.9+ KB


----

## 1. Tratamento dos Dados

In [37]:
df.head()

,Ad_ID,Campaign_Name,Clicks,Impressions,Cost,Leads,Conversions,Conversion Rate,Sale_Amount,Ad_Date,Device,Keyword
0,A1000,campanha_1,104.0,4498.0,231.88,14.0,7.0,0.058,1892.0,2024-11-16,desktop,learn_data_analytics
1,A1001,campanha_1,173.0,5107.0,216.84,10.0,8.0,0.046,1679.0,20-11-2024,mobile,data_analytics_course
2,A1002,campanha_2,90.0,4544.0,203.66,26.0,9.0,NaN,1624.0,2024/11/16,desktop,data_analitics_online
3,A1003,campanha_3,142.0,3185.0,237.66,17.0,6.0,NaN,1225.0,2024-11-26,tablet,data_anaytics_training
4,A1004,campanha_4,156.0,3361.0,195.90,30.0,8.0,NaN,1091.0,2024-11-22,desktop,online_data_analytic


In [25]:
# visualização das categorias
for col in df.columns:
    print(col.upper())
    print(df[col].value_counts())
    print('_'*50)

AD_ID
A1000    1
A2746    1
A2728    1
A2729    1
A2730    1
        ..
A1868    1
A1869    1
A1870    1
A1871    1
A3599    1
Name: Ad_ID, Length: 2600, dtype: int64
__________________________________________________
CAMPAIGN_NAME
campanha_3    680
campanha_4    647
campanha_1    637
campanha_2    636
Name: Campaign_Name, dtype: int64
__________________________________________________
CLICKS
127.0    34
148.0    32
89.0     29
146.0    28
151.0    28
         ..
107.0    13
129.0    13
159.0    12
161.0    12
158.0    11
Name: Clicks, Length: 120, dtype: int64
__________________________________________________
IMPRESSIONS
4653.0    6
4441.0    5
5509.0    5
5545.0    5
3948.0    5
         ..
4224.0    1
3579.0    1
3188.0    1
5493.0    1
5453.0    1
Name: Impressions, Length: 1702, dtype: int64
__________________________________________________
COST
$246.3     4
$218.69    4
$219.15    4
$245.13    4
$249.33    4
          ..
$223.28    1
$221.65    1
$214.29    1
$191.68    1
$220.

**Observações:**
- Notamos que na variável `Campaing_Name` as 4 campanhas possuem o mesmo nome, porém com escrituras diferentes. Apesar do mesmo nome, não devem ser a mesma campanha e sim campanhas distintas, afinal nosso trabalho é fazer a análise de qual tem a melhor perfomance. Dito isso, a forma que optarei para corrigir será substituindo os nomes por "Campanha 1", "Campanha 2" e sucessivamente.
- Em `Location` vemos que a localização é no mesmo lugar em todas as campanhas: Hyderabad (Capital e maior cidade do estado indiano de Tengala). Sabendo disso, ao invés de tratar os caracteres da coluna, podemos simplesmente remove-la já que se aplica a todos os registros e não tem variância, não adiciona informação para a análise de performance, além de economizar memória e torna o conjunto de dados mais limpo.
- O tipo de dispositivo em `Device` está com valores diferentes para cada categoria, poderemos resolver corrigindo os caracteres e agregando-os.
- As keywords parecem não haver um ádrão definido, porém é uma estratégia de marketing testar combinações diferentes de palavras-chave, para ver se há impacto sobre as campanhas. O que podemos fazer futuramente antes de treinar um modelo de machine learning, é separar cada uma das palavras em variáveis diferentes para ver se uma tem mais impacto que outra.


**Removendo a coluna Location**

In [15]:
df.drop(columns='Location', axis=1, inplace=True)

**Tratando o Nome da Campanha**

In [14]:
# Listando os valores únicos
campanha_unic = df['Campaign_Name'].unique()

# criando um dicionário de mapeamento
mapa = {
    campanha_unic[0]: 'campanha_1',
    campanha_unic[1]: 'campanha_2',
    campanha_unic[2]: 'campanha_3',
    campanha_unic[3]: 'campanha_4'
}

# aplicando a substituição
df['Campaign_Name'] = df['Campaign_Name'].map(mapa)

df['Campaign_Name'].value_counts()

campanha_3    680
campanha_4    647
campanha_1    637
campanha_2    636
Name: Campaign_Name, dtype: int64

**Tratando os Tipos de Dispsitivos**

In [ ]:
# padroniza os caracteres em minusculo
df['Device'] = df['Device'].apply(lambda x: x.lower())

In [20]:
df.Device.value_counts()

desktop    888
mobile     878
tablet     834
Name: Device, dtype: int64

**Tratando Keywords**

In [ ]:
# substitui os espaçamentos em cadacategoria por "_"
df['Keyword'] = df['Keyword'].apply(lambda x: x.replace(' ', '_'))

 **Tratando Inconcistências em Variáveis Financeiras**

In [ ]:
# lista as variáveis financeiras
fin = ['Cost', 'Sale_Amount']

# itera sobre cada uma das variáveis
for col in df[fin]:
    # remove caractere "$"
    df[col] = df[col].str.replace('$', '', regex=False)
    # formata os valores numéricos para o tipo float
    df[col] = df[col].astype(float)

**Corrigindo variável Data**

- Algumas datas estão no formato YYYY-MM-DD, outras no formato DD-MM-YYYY.
- Precisamos primeiro padronizar para um único formato e depois converter para o `datetime` do Pandas.

In [ ]:
# tentando converter diretamente todos os valores
df['Ad_Date'] = pd.to_datetime(df['Ad_Date'], errors='coerce', dayfirst=False)

# identificando datas que falharam
erros = df[df['Ad_Date'].isna()]

# para as datas que fallharam, tentar converter novamente
df.loc[df['Ad_Date'].isna(), 'Ad_Date'] = pd.to_datetime(
    df.loc[df['Ad_Date'].isna(), 'Ad_Date'], errors='coerce', dayfirst=True
)

# conferindo se ainda há falhas
df[df['Ad_Date'].isna()]

,Ad_ID,Campaign_Name,Clicks,Impressions,Cost,Leads,Conversions,Conversion Rate,Sale_Amount,Ad_Date,Device,Keyword


**Tratando Valores Nulos**

In [47]:
df.isnull().sum()

Ad_ID                0
Campaign_Name        0
Clicks             112
Impressions         54
Cost                97
Leads               48
Conversions         74
Conversion Rate    626
Sale_Amount        139
Ad_Date              0
Device               0
Keyword              0
dtype: int64

**Observações:**
- Possuímos valores nulos em: Cliques, impressões, custo, leads, conversões, taxa de conversão, valor da venda.
- 112 anúncios estão com dados nulos em cliques, faz sentido? Pode ser que esses valores ausentes não sejam por inconsistência, e sim por que não geraram cliques mesmo. Podemos substituir os valores nulos por "0" e posteriormente fazer uma análise das campanhas que não geraram cliques e abstrair algum insight.
- Temos também 54 anúncios com impressões nulas, por que? Seria devido ao fato de que realmente não houveram as impressões ou apenas inconcistência? Faz sentido para o negócio? Assim como em Cliques, iremos substituir por "0" para uma análise futura.
- No caso dos nulos em Custo do anúncio, seria tambem o fator inconsistência? Pode ser anúncios em fase de espera de aprovação, problemas de pagamento ou até mesmo não ter gerado nenhum resultado mensurável.
- As mesmas questões acima se aplicam para a variável de Conversão.
- 626 nulos em Taxa de Conversão aparenta ser estranho, principalmente em um conjunto de dados com apenas 2600 registros. Aqui pode haver de fato uma inconcistência nos dados.
- Em Sale_Amount temos 139 nulos, mas como essa variável determina a receita gerada após a conversão, podemos supor que as conversões em específico não foram de vendas, e talvez Leads.

Antes de remover qualquer valor desses, iremos conferir se de fato é uma inconcistência ou apenas falta de tratamento.